In [1]:
import fastf1 as f1
import pandas as pd
import numpy as np
import datetime
import os
from fastf1.core import Laps,Session,BaseDataFrame

In [2]:
f1.Cache.enable_cache('cache\\')
seasons = [2023,2024]
drop_labels=['Time','PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
                'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
                'FreshTyre', 'LapStartTime',
                'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
                'FastF1Generated', 'IsAccurate']

In [3]:
def update_cache(_year:int):
    f1.Cache.offline_mode(enabled=False)
    e1 = pd.DataFrame(data=f1.get_event_schedule(_year))
    e1 = e1[e1['RoundNumber']!=0]
    for es in e1['EventName']:
        e2 = f1.get_session(_year,gp=es,identifier='R')
        if (e2.date.date()<datetime.date.today()):
                e2.load()

In [48]:
def update_user_data(_season:int):
    df_train=pd.DataFrame()

    for season in _season:
        e1 = pd.DataFrame(data=f1.get_event_schedule(season))
        e1 = e1[e1['RoundNumber']!=0]
        for es in e1['EventName']:
            e2 = f1.get_session(season,es,'R')
            if (e2.date.date()<datetime.date.today()):
                e2.load()
                laps = e2.laps.reset_index(drop=True)
                weather = e2.laps.get_weather_data().reset_index(drop=True)
                data = pd.concat([laps,weather],axis=1)
                data['Year'] = e2.event.EventDate.year
                data['GrandPrix'] = e2.event.EventName
                data['LapTime'] = data['LapTime'].dt.total_seconds()
                df_test = pd.DataFrame()
                for drv in data['Driver'].unique():
                    temp = data[data['Driver']==drv].iloc[0:int(0.7*len(data['LapNumber'].unique()))]
                    df_train = pd.concat([df_train,temp])
                
                    if (os.path.exists('test\\'+str(season)+' '+es+'.csv')==False):
                        temp = data[data['Driver']==drv].iloc[int(0.7*len(data['LapNumber'].unique()))+1:]
                        df_test = pd.concat([df_test,temp])
                df_test.drop(labels=drop_labels,axis=1,inplace=True)
                df_test.to_csv('test\\'+str(season)+' '+es+'.csv',index=None)
    
    df_train.drop(labels=drop_labels,axis=1,inplace=True)
    df_train.reset_index(drop=True,inplace=True)
    df_train.to_csv('train\\'+str(_season)+'.csv',index=None)

In [28]:
update_cache(2024)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.8]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix

In [49]:
update_user_data([2023,2024])

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.8]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Pri